<a href="https://colab.research.google.com/github/FirySnehasis/SnT-Secy-Task/blob/main/SnT%20Task%201%20Strategy%20Implementation%20and%20Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install vectorbt
import pandas as pd
import numpy as np
import yfinance as yf
import math
import matplotlib.pyplot as plt
import vectorbt as vbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
btcdata = yf.download('BTC-USD', start='2021-01-01', end='2023-06-15')
ethdata = yf.download('ETH-USD', start='2021-01-01', end='2023-06-15')

btcdata['Daily Returns'] = 100*(btcdata['Close'] - btcdata['Open'])/(btcdata['Open'])
ethdata['Daily Returns'] = 100*(ethdata['Close'] - ethdata['Open'])/(ethdata['Open'])
btcdata = btcdata.reset_index()
ethdata = ethdata.reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
Corrcoef = np.corrcoef(ethdata['Close'], btcdata['Close'])[0,1]
print('The Correlation Coefficient is : '+str(Corrcoef))
print("It shows there exits a positive and a strong relationship between NASDAQ and NSE")

The Correlation Coefficient is : 0.7823295378983014
It shows there exits a positive and a strong relationship between NASDAQ and NSE


In [5]:
############# Let us assume that ETH lags
count = 0
corr1=[]
max1=0
index1=0
corr2=[]
max2=0
index2=0
for i in range(0,23):
  corr1 = ethdata['Close'].corr(btcdata['Close'].shift(i))
  if(ethdata['Close'].corr(btcdata['Close'].shift(i)) >= Corrcoef and ethdata['Close'].corr(btcdata['Close'].shift(i)) > max1):
    max1 = ethdata['Close'].corr(btcdata['Close'].shift(i))
    index1 = i
    count+=1
########### Let us assume that BTC lags
if(count==0):
  for j in range(0,20):
    corr2 = btcdata['Close'].corr(ethdata['Close'].shift(j))
    if(btcdata['Close'].corr(ethdata['Close'].shift(j)) >= Corrcoef and btcdata['Close'].corr(ethdata['Close'].shift(j)) > max2):
      max2 = btcdata['Close'].corr(ethdata['Close'].shift(j))
      index2 = j
      count+=1
print("BTC leads ETH by "+str(index1))
print("ETH leads BTC by "+str(index2))

BTC leads ETH by 22
ETH leads BTC by 0


In [6]:
btcdata['Returns'] = btcdata['Daily Returns'].shift(22)
btcdata['Returns'].fillna(0, inplace=True)
# If Bitcoin's return is positive, we go long on Ethereum, else we go short
ethdata['Position'] = np.where(btcdata['Returns'].shift(1) > 0, 1, -1)

price = ethdata['Close']
signals = pd.DataFrame({
    'Entry': ethdata['Position'].replace(-1, 0),
    'Exit': ethdata['Position'].replace(1, 0)
})

portfolio = vbt.Portfolio.from_signals(price, signals['Entry'], signals['Exit'], init_cash=10000000, fees=0.001, freq='D')

print(portfolio.stats())


Start                                                 0
End                                                 894
Period                                895 days 00:00:00
Start Value                                  10000000.0
End Value                               16447583.016955
Total Return [%]                               64.47583
Benchmark Return [%]                          125.98475
Max Gross Exposure [%]                            100.0
Total Fees Paid                          9411441.825609
Max Drawdown [%]                              61.066731
Max Drawdown Duration                 583 days 00:00:00
Total Trades                                        232
Total Closed Trades                                 231
Total Open Trades                                     1
Open Trade PnL                            -16447.583017
Win Rate [%]                                   49.78355
Best Trade [%]                                28.073499
Worst Trade [%]                              -12

In [7]:
# Plot the total profit and loss of the portfolio over time
portfolio.plot().show()

# Plot the price and trades
portfolio.plot_trades().show()

In [8]:
ethdata['Buy_Signal'] = 0
ethdata['Sell_Signal'] = 0
ethdata['MACD'] = ethdata['Close'].ewm(span=12, adjust=False).mean() - ethdata['Close'].ewm(span=26, adjust=False).mean()
ethdata['Signal Line MACD'] = ethdata['MACD'].ewm(span=9, adjust=False).mean()
for i in range(1,894):
  if(0>=ethdata.loc[i,'MACD'] and ethdata.loc[i+1,'MACD']>=0):
    ethdata.iloc[i, 9] = 1

  elif(0<=ethdata.loc[i,'MACD'] and ethdata.loc[i+1,'MACD']<=0):
    ethdata.iloc[i, 10] = -1

portfolio1 = vbt.Portfolio.from_signals(ethdata['Close'], ethdata['Buy_Signal'], ethdata['Sell_Signal'], init_cash=10000000, fees=0.001, freq='D')
print(portfolio1.stats())
portfolio1.plot().show()
portfolio1.plot_trades().show()

Start                                         0
End                                         894
Period                        895 days 00:00:00
Start Value                          10000000.0
End Value                       25864137.629023
Total Return [%]                     158.641376
Benchmark Return [%]                  125.98475
Max Gross Exposure [%]                    100.0
Total Fees Paid                    739302.43941
Max Drawdown [%]                      41.810054
Max Drawdown Duration         305 days 00:00:00
Total Trades                                 17
Total Closed Trades                          17
Total Open Trades                             0
Open Trade PnL                              0.0
Win Rate [%]                          47.058824
Best Trade [%]                        48.828456
Worst Trade [%]                       -24.90703
Avg Winning Trade [%]                 23.561164
Avg Losing Trade [%]                  -6.914364
Avg Winning Trade Duration     41 days 1